<a href="https://colab.research.google.com/github/Monterazo/Ai-Videomaker/blob/main/talegenius.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creating tales with Artificial Inteligence


>[Creating tales with Artificial Inteligence](#scrollTo=UXa9IwkeokWH)

>[Setup](#scrollTo=ep0A2pLDnoWK)

>[Authenticate](#scrollTo=8BCZiwH6cLnv)

>[Run Script](#scrollTo=v35GZfs_QyUI)

>[Run images](#scrollTo=Ax6xbVZOpnaV)

>[Run Audio](#scrollTo=GQ80oLjXcdGP)



# Setup

To run this notebook, you’ll need to create a [Replicate](https://replicate.com) account and install the Replicate python client.

In [ ]:
# install replicate client
!pip install replicate
!pip install -q -U google-generativeai


In [ ]:
#import necessary packages
import google.generativeai as genai
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown
from google.colab import userdata


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Authenticate
Before running any Python scripts that use the API, you need to set your Replicate API and Gemini tokens in your environment.




In [ ]:
from getpass import getpass
import os
import google.generativeai as genai

# Replicate
# get a token: https://replicate.com/account
REPLICATE_API_TOKEN = getpass("Replicate Key:")
os.environ["REPLICATE_API_TOKEN"] = REPLICATE_API_TOKEN

#Google
# get a token: https://ai.google.dev/
GOOGLE_API_KEY=getpass('Gemini Key:')

genai.configure(api_key=GOOGLE_API_KEY)
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY



Replicate Key:··········
Gemini Key:··········


# Run Script
The following code runs Google-Gemini-1.0-pro to generate the script for our story.


In [ ]:
import google.generativeai as genai

model = genai.GenerativeModel('gemini-pro')
response = model.generate_content(input('Enter the theme: ')+ "is the theme of my three part . Each part must have a short Narration and a Prompt to generate an image about the event narrated")

to_markdown(response.text)




Enter the theme: Red Velvet cake factory


> **Part 1: The Enchanted Oven**
> 
> *Narration:* In the heart of a bustling city, nestled amidst the sweet scents of baked goods, stood a remarkable factory known as the Red Velvet Cake Factory. This factory was no ordinary establishment; it possessed a magical charm that could transport anyone who stepped inside to a world where imagination and deliciousness intertwined.
> 
> *Prompt:* Create a vibrant and whimsical digital painting that captures the essence of the Red Velvet Cake Factory. Let the vibrant red and white hues dance across the canvas, while golden lights illuminate the factory's majestic facade. In the background, hint at the delectable treats that await inside, such as towering red velvet cakes adorned with intricate swirls of cream cheese frosting.
> 
> **Part 2: The Secret Recipe**
> 
> *Narration:* Within the factory's walls, a team of dedicated bakers embarked on a daily quest to create the most delectable red velvet cakes the world had ever tasted. They held the secret recipe close to their hearts, passed down from generations of master bakers. It was a recipe that whispered tales of love, laughter, and the magic of baking.
> 
> *Prompt:* Craft a mesmerizing digital painting that offers a glimpse into the inner sanctum of the Red Velvet Cake Factory's kitchen. Show the bakers with their flour-dusted aprons and wide smiles, carefully measuring ingredients and whisking batter. Let the warmth of the oven's glow bathe the scene in a golden light, and hint at the heavenly aroma that fills the air.
> 
> **Part 3: The Sweetest Symphony**
> 
> *Narration:* As the final cakes emerged from the oven, the factory transformed into a symphony of sweet melodies. The clinking of mixing bowls, the gentle hum of the machinery, and the laughter of the bakers blended together, creating an enchanting harmony that echoed through the corridors. It was a symphony that celebrated the joy of baking and the love that was poured into each and every red velvet cake.
> 
> *Prompt:* Design an enchanting digital painting that captures the harmonious spirit of the Red Velvet Cake Factory. Depict the bakers dancing and singing amidst towering stacks of red velvet cakes, their faces radiant with joy. Let the music notes swirl around them like confetti, adding a touch of whimsy to the scene. In the background, hint at the eager customers waiting outside, their eyes sparkling with anticipation for a taste of the factory's legendary treats.

In [ ]:
#Generate Image 1

Image_1 = model.generate_content("I have an story and I want only the Image Prompt for the part 1 with, the story is:" + response.text)
Image_2 = model.generate_content("I have an story and I want only the Image Prompt for the part 2 with, the story is:" + response.text)
Image_3 = model.generate_content("I have an story and I want only the Image Prompt for the part 3 with, the story is:" + response.text)

to_markdown(Image_1.text)

> **Image Prompt for Part 1: The Enchanted Oven**
> 
> Create a vibrant and whimsical digital painting that captures the essence of the Red Velvet Cake Factory. In the heart of a bustling city, a remarkable factory stands amidst the sweet scents of baked goods. Its red and white facade is illuminated by golden lights, creating a magical charm. Towering red velvet cakes adorned with intricate swirls of cream cheese frosting hint at the delectable treats that await inside. Let the vibrant hues and golden lights dance across the canvas, inviting viewers into a world where imagination and deliciousness intertwine.

# Run images
The following example runs [stability-ai/stable-diffusion](https://replicate.com/stability-ai/stable-diffusion) and [stability-ai/stable-video-diffusion](https://replicate.com/stability-ai/stable-video-diffusion?input=python):

In [ ]:
import replicate

output = replicate.run(
  "stability-ai/sdxl:39ed52f2a78e934b3ba6e2a89f5b1c712de7dfea535525255b1aa35c5565e08b",
  input={"prompt": response.text}
)

output

from IPython.display import Image
Image(url=output[0])

In [ ]:
from urllib.request import urlretrieve

model = replicate.models.get("stability-ai/stable-video-diffusion")
version = model.versions.get("3f0457e4619daac51203dedb472816fd4af51f3149fa7a9e0b5ffcf1b8172438")

prediction = replicate.predictions.create(
    version=version,
    input={
      "cond_aug": 0.02,
      "decoding_t": 7,
      "input_image": output[0],
      "video_length": "25_frames_with_svd_xt",
      "sizing_strategy": "maintain_aspect_ratio",
      "motion_bucket_id": 127,
      "frames_per_second": 5
    })
prediction.wait()

urlretrieve(prediction.output, "/tmp/out.mp4")

from IPython.display import Video

Video(prediction.output)

#Run Audio
API to Run the narration audio

#Compile Video
